<a href="https://colab.research.google.com/github/m-newhauser/rep-or-dem-tweets/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.6.0
!pip install tweet-preprocessor


In [2]:
import random
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import preprocessor as p

from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import (
    TFDistilBertForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

random.seed(123)

In [4]:
# Load data from fivethirtyeight
tweets = pd.read_csv("senators_training.csv")
tweets_validation = pd.read_csv("senators_validation.csv")

# Remove numbers, emojis and &'s
p.set_options(p.OPT.NUMBER, p.OPT.EMOJI)
tweets["text_clean"] = tweets["text"].apply(p.clean)
tweets["text_clean"] = tweets["text_clean"].str.replace("&amp;", "and ")
tweets_validation["text_clean"] = tweets_validation["text"].apply(p.clean)
tweets_validation["text_clean"] = tweets_validation["text_clean"].str.replace("&amp;", "and ")

# Truncate tweets to max 512
tweets["text_clean"] = tweets["text_clean"].str[:512]
tweets_validation["text_clean"] = tweets_validation["text_clean"].str[:512]


In [17]:
# Create a column with numeric labels
label_mapping = {
    "D": 0,
    "R": 1
}

tweets['label'] = np.where(tweets['party'] == "D", 0, 1)
tweets_validation['label'] = np.where(tweets_validation['party'] == "D", 0, 1)

In [20]:
# Convert to list
texts = list(tweets.text)
labels = list(tweets.label)
val_texts = list(tweets_validation.text)
val_labels = list(tweets_validation.label)

# Split training dataset into test and train
(train_texts, test_texts, train_labels, test_labels) = train_test_split(
    texts, labels, test_size=0.3
)

In [21]:
print(type(train_texts))
print(type(train_labels))

print(train_texts[0])
print(train_labels[0])

<class 'list'>
<class 'list'>
Corker on @CNN: Bipartisan majority in Congress and vast majority of the American people oppose the #BadIranDeal: http://t.co/VaHTJ2RoGH.
1


In [22]:
# Load DistilBERT tokenizer and tokenize (encode) the texts
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


In [23]:
print(type(train_encodings))
print(train_encodings[10])

<class 'transformers.tokenization_utils_base.BatchEncoding'>
Encoding(num_tokens=63, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [41]:
# Wrap encodings in a Tensor Flow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

# Learning rate, epochs, batch size

# Set the training arguments
training_args = TFTrainingArguments(
    output_dir="./results",  # output directory
    num_train_epochs=6,  # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir="./logs",  # directory for storing logs
    logging_steps=10,
)

with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", 
        num_labels=2
    )

trainer = TFTrainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=val_dataset,  # evaluation dataset
)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_99', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [42]:
# Train the model
trainer.train()

In [43]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 0.6836820125579834}

In [44]:
# Get predictions
predictions = trainer.predict(val_dataset)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

compute_metrics(predictions)

{'accuracy': 0.65,
 'f1': 0.6708463949843261,
 'precision': 0.6331360946745562,
 'recall': 0.7133333333333334}